In [1]:
import sqlite3
from pathlib import Path
import numpy as np

In [2]:
# path to databases
print("For which experiment would you get the graph elements for coordinates?")
experiment = int(input())
if experiment == 1:
    db_path = Path('E:/HumanA/Data/DataBase/HumanA_Exp1.db')
    #db_path = Path('E:/HumanA/Data/HumanA_Exp1_WorkingData.db')
elif experiment == 2:
    db_path = Path('E:/HumanA/Data/DataBase/HumanA_Exp2.db')
    #db_path = Path('E:/HumanA/Data/HumanA_Exp2_WorkingData.db')
if not db_path or not db_path.exists():
    db_path = ':memory:'

For which experiment would you get the graph elements for coordinates?


In [3]:
# connect to database
connection=sqlite3.connect(db_path)
cr=connection.cursor()

In [4]:
def create_reduced_avatar_table():
    sql_instruction = """CREATE TABLE IF NOT EXISTS "avatars_reduced"(
        Id INTEGER NOT NULL UNIQUE,
        AvatarId NUMERIC NOT NULL,
        AvatarName TEXT,
        AvatarPositionGlobal_x NUMERIC,
        AvatarPositionGlobal_y NUMERIC,
        AvatarPositionGlobal_z NUMERIC,
        AvatarCategory NUMERIC,
        coor_converted_x NUMERIC,
        coor_converted_z NUMERIC,
        coor_converted_precise_x NUMERIC,
        coor_converted_precise_z NUMERIC,
        graph_element_type TEXT,
        node NUMERIC,
        edge_start NUMERIC,
        edge_end NUMERIC,
        PRIMARY KEY ("Id" AUTOINCREMENT)
        FOREIGN KEY(AvatarId) REFERENCES avatars(AvatarId)
        FOREIGN KEY(node) REFERENCES graph_coordinates(nodeNr)
        FOREIGN KEY(edge_start) REFERENCES graph_coordinates(edgeStart)
        FOREIGN KEY(edge_end) REFERENCES graph_coordinates(edgeEnd)
        );
        """
    cr.execute(sql_instruction)
    connection.commit()

In [5]:
def get_avatar_information():
    sql_instruction = f"""SELECT AvatarID, AvatarName, AvatarPositionGlobal_x,  AvatarPositionGlobal_y, 
        AvatarPositionGlobal_z, Category FROM avatars"""
    cr.execute(sql_instruction)
    avatars = cr.fetchall()
    return avatars

In [6]:
#def getMinMaxValues():
#    # select xMin
#    sql_instruction = """
#    SELECT MIN(playerBodyPosition_x) FROM data_points;
#    """
#    cr.execute(sql_instruction)
#    xMin = cr.fetchone()
#    xMin = round(xMin[0],2)
#
#    # select xMax
#    sql_instruction = """
#    SELECT MAX(playerBodyPosition_x) FROM data_points;
#    """
#    cr.execute(sql_instruction)
#    xMax = cr.fetchone()
#    xMax = round(xMax[0],2)
#
#    # select zMin
#    sql_instruction = """
#    SELECT MIN(playerBodyPosition_z) FROM data_points;
#    """
#    cr.execute(sql_instruction)
#    zMin = cr.fetchone()
#    zMin = round(zMin[0],2)
#
#    # select zMax
#    sql_instruction = """
#    SELECT MAX(playerBodyPosition_z) FROM data_points;
#    """
#    cr.execute(sql_instruction)
#    zMax = cr.fetchone()
#    zMax = round(zMax[0],2)
#
#    return xMin, xMax, zMin, zMax

In [7]:
def convert_coordinate(coordinate, meters):
    step = meters
    x = int((np.abs(xMin) + coordinate[0]) / step)
    y = int((np.abs(zMin) + coordinate[1]) / step)
    return x, y

In [8]:
def convert_coordinate_more_precise(coordinate, meters):
    step = meters
    x = (np.abs(xMin) + coordinate[0]) / step
    y = (np.abs(zMin) + coordinate[1]) / step
    return x, y

In [9]:
create_reduced_avatar_table()

In [10]:
def get_min_dist_nodes(x_coor, z_coor):
    
    sql_instruction = f"""SELECT DISTINCT graph_element_type, nodeNr, nodeCentroid_x, nodeCentroid_z, nodeRadius, ABS(({x_coor} - graph_coordinates.nodeCentroid_x)*({x_coor} - graph_coordinates.nodeCentroid_x) + 
            ({z_coor} - graph_coordinates.nodeCentroid_z)*({z_coor} - graph_coordinates.nodeCentroid_z)) AS Distance
        FROM graph_coordinates 
        WHERE ABS(({x_coor} - graph_coordinates.nodeCentroid_x)*({x_coor} - graph_coordinates.nodeCentroid_x) + 
                ({z_coor} - graph_coordinates.nodeCentroid_z)*({z_coor} - graph_coordinates.nodeCentroid_z)) = (
                SELECT MIN(ABS(({x_coor} - graph_coordinates.nodeCentroid_x)*({x_coor} - graph_coordinates.nodeCentroid_x) + 
                    ({z_coor} - graph_coordinates.nodeCentroid_z)*({z_coor} - graph_coordinates.nodeCentroid_z))) 
                FROM graph_coordinates
                WHERE ABS(({x_coor} - graph_coordinates.nodeCentroid_x)*({x_coor} - graph_coordinates.nodeCentroid_x) + 
                ({z_coor} - graph_coordinates.nodeCentroid_z)*({z_coor} - graph_coordinates.nodeCentroid_z)) <= graph_coordinates.nodeRadius*graph_coordinates.nodeRadius)"""
    cr.execute(sql_instruction)
    minNodes = cr.fetchall()
    if len(minNodes) == 1:
        min_node = minNodes[0]
    else:
        min_node = []
    return min_node

In [11]:
def get_min_dist_edges(x_coor, z_coor):
    
    sql_instruction = f"""SELECT DISTINCT graph_element_type, edgeStart,edgeEnd, X_Coor, Z_Coor, ABS(({x_coor} - graph_coordinates.X_Coor)*
                        ({x_coor} - graph_coordinates.X_Coor) + ({z_coor} - graph_coordinates.Z_Coor)*
                        ({z_coor} - graph_coordinates.Z_Coor)) AS Distance
        FROM graph_coordinates 
        WHERE ABS(({x_coor} - graph_coordinates.X_Coor)*({x_coor} - graph_coordinates.X_Coor) + 
                ({z_coor} - graph_coordinates.Z_Coor)*({z_coor} - graph_coordinates.Z_Coor)) = (
                SELECT MIN(ABS(({x_coor} - graph_coordinates.X_Coor)*({x_coor} - graph_coordinates.X_Coor) + 
                    ({z_coor} - graph_coordinates.Z_Coor)*({z_coor} - graph_coordinates.Z_Coor))) 
                FROM graph_coordinates
                WHERE graph_element_type = 'Edge')"""
    #print(sql_instruction)
    cr.execute(sql_instruction)
    minEdges = cr.fetchall()
    if len(minEdges) == 1:
        minEdge = minEdges[0]
    else:
        minEdge = []
    return minEdge

In [12]:
def getClosestGraphElement(xCoor,zCoor):
    closestElement = get_min_dist_nodes(xCoor,zCoor)
    if closestElement == []:
        closestElement = get_min_dist_edges(xCoor,zCoor)
    return closestElement 

In [13]:
avatars = get_avatar_information()
xMin, xMax, zMin, zMax = -442.87, 439.76, -280.14, 301.68
for avatar in avatars:
    avatarId, avatarName, GlobalxCoor, GlobalyCoor, GlobalzCoor, Category = avatar
    
    coor_conv = convert_coordinate([GlobalxCoor,GlobalzCoor], 4)
    coor_conv_prec = convert_coordinate_more_precise([GlobalxCoor,GlobalzCoor], 4)
    
    closestElement = getClosestGraphElement(coor_conv_prec[0], coor_conv_prec[1])
    if closestElement != []:
        element_type = closestElement[0] 
        if element_type == 'Node':
            node = closestElement[1]
            values = str((avatarId,avatarName,GlobalxCoor, GlobalyCoor, GlobalzCoor, Category, coor_conv[0], 
              coor_conv[1], coor_conv_prec[0], coor_conv_prec[1], element_type,node))
            sql_instruction = f"""INSERT INTO avatars_reduced (AvatarId, AvatarName, AvatarPositionGlobal_x, 
                        AvatarPositionGlobal_y, AvatarPositionGlobal_z, AvatarCategory, coor_converted_x, 
                        coor_converted_z, coor_converted_precise_x, coor_converted_precise_z, graph_element_type, node)
                        VALUES {values};"""
        elif element_type == 'Edge':
            edgeStart = closestElement[1]
            edgeEnd = closestElement[2]
            values = str((avatarId,avatarName,GlobalxCoor, GlobalyCoor, GlobalzCoor, Category, coor_conv[0], 
              coor_conv[1], coor_conv_prec[0], coor_conv_prec[1], element_type,edgeStart,edgeEnd))
            sql_instruction = f"""INSERT INTO avatars_reduced (AvatarId, AvatarName, AvatarPositionGlobal_x, 
                        AvatarPositionGlobal_y, AvatarPositionGlobal_z, AvatarCategory, coor_converted_x, 
                        coor_converted_z, coor_converted_precise_x, coor_converted_precise_z, graph_element_type, 
                        edge_start,edge_end)
                        VALUES {values};"""
            #print(sql_instruction)
    else:
        values = str((avatarId,avatarName,GlobalxCoor, GlobalyCoor, GlobalzCoor, Category, coor_conv[0], 
                      coor_conv[1], coor_conv_prec[0], coor_conv_prec[1]))

        sql_instruction = f"""INSERT INTO avatars_reduced (AvatarId, AvatarName, AvatarPositionGlobal_x, 
                            AvatarPositionGlobal_y, AvatarPositionGlobal_z, AvatarCategory, coor_converted_x, 
                            coor_converted_z, coor_converted_precise_x, coor_converted_precise_z)
                            VALUES {values};"""
    cr.execute(sql_instruction)
connection.commit()
connection.close()

In [14]:
connection.close()